In [56]:
import sys
import os

# Add the parent directory (project root) relative to the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

Current working directory: c:\Users\cjsta\git\inverse_kinematics_prediction\notebooks
Project root added to sys.path: c:\Users\cjsta\git\inverse_kinematics_prediction


In [57]:
from utils.robot_arm import RobotArm
from utils.ik_solver import DampedLeastSquaresSolver, JacobianTransposeSolver, PseudoInverseSolver
import numpy as np

# Create a 4-DOF robot arm with 5 links, each of length 1.0 unit.
num_joints = 4
joint_limits = [(-1*np.pi, 1*np.pi), (-0.8*np.pi, 0.8*np.pi), (-0.8*np.pi, 0.8*np.pi), (-0.8*np.pi, 0.8*np.pi)]
rotation_axes = ['z', 'y', 'y', 'y']
link_lengths = [0.2, 0.2, 0.5, 0.5, 0.2]
link_axes = ['z', 'z', 'z', 'z', 'z']

# Instantiate the RobotArm object
robot_arm = RobotArm(num_joints=num_joints, 
                     joint_limits=joint_limits, 
                     link_lengths=link_lengths, 
                     rotation_axes=rotation_axes,
                     link_axes=link_axes)

# Create a solver
dls_solver = DampedLeastSquaresSolver(robot_arm, damping_factor=0.1)
jt_solver = JacobianTransposeSolver(robot_arm, step_size=0.05)
p_solver = PseudoInverseSolver(robot_arm)

# Solve IK
target_position = np.array([0.16868597, -0.03597471,  1.00171875])
dls_joint_angles, dls_computation_time, dls_success = dls_solver.solve(target_position)
jt_joint_angles, jt_computation_time, jt_success = jt_solver.solve(target_position)
p_joint_angles, p_computation_time, p_success = p_solver.solve(target_position)

print(f"\nTarget Position: {target_position}")

print(f"\nDamped Least Squares Solver")
print(f"\tSuccess: {dls_success}")
print(f"\tComputation Time: {dls_computation_time:.4f} seconds")
print(f"\tJoint Angles: {dls_joint_angles}")
print(f"\tEnd Effector Position: {robot_arm.forward_kinematics(dls_joint_angles)}")

print(f"\nJacobian Transpose Solver")
print(f"\tSuccess: {jt_success}")
print(f"\tComputation Time: {jt_computation_time:.4f} seconds")
print(f"\tJoint Angles: {jt_joint_angles}")
print(f"\tEnd Effector Position: {robot_arm.forward_kinematics(jt_joint_angles)}")

print(f"\nPseudo Inverse Solver")
print(f"\tSuccess: {p_success}")
print(f"\tComputation Time: {p_computation_time:.4f} seconds")
print(f"\tJoint Angles: {p_joint_angles}")
print(f"\tEnd Effector Position: {robot_arm.forward_kinematics(p_joint_angles)}")


Target Position: [ 0.16868597 -0.03597471  1.00171875]

Damped Least Squares Solver
	Success: True
	Computation Time: 0.0013 seconds
	Joint Angles: [-0.21148502 -0.77893074  1.56715423  1.34526164]
	End Effector Position: (array([ 0.16861393, -0.03620064,  1.00169316]), (np.float64(0.09241905918002832), np.float64(0.8707410089454436), np.float64(-0.05097630097678012), np.float64(0.4802814066562166)))

Jacobian Transpose Solver
	Success: False
	Computation Time: 0.0194 seconds
	Joint Angles: [-2.95373849  0.32482198  2.12815919 -2.51327412]
	End Effector Position: (array([-0.45706653, -0.08688631,  0.68742581]), (np.float64(0.030009048431719792), np.float64(0.0028269807718091953), np.float64(0.9951397257943848), np.float64(-0.09374642040000791)))

Pseudo Inverse Solver
	Success: True
	Computation Time: 0.0140 seconds
	Joint Angles: [-0.21011664 -0.3277448   1.86438257 -2.51327412]
	End Effector Position: (array([ 0.16922465, -0.03608959,  1.00242389]), (np.float64(-0.049196581261098195